# Explanation

This is the first script run.

The data originally came as full periapical radiographs, which contain 3-4 teeth typically. This code is to separate them into individual teeth and classify them based on their roots.

The clinicians (A Orishko and H J Tan) labelled the landmarks, type of tooth, bounding box and root morphology of each individual tooth.

As is expected with large labelling-based projects, there were some errors in the labelling. One of the key functions of this script is to identify these errors and exclude them from the processed dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/Labelled_Data.zip' #placeholder for github

In [ ]:
!mkdir 'individual_teeth'
!mkdir 'individual_teeth/single_roots'
!mkdir 'individual_teeth/double_roots'
!mkdir 'individual_teeth/triple_roots'

In [ ]:
import json
import sys
import numpy as np
import cv2
import glob
from google.colab.patches import cv2_imshow #specifically for collab
import pandas as pd

In [ ]:
def add_smart_buffer(image,bbox,buffer): #adds a buffer to the cropped image that does not go outside of the image boundary (causing errors)

  img_height, img_width,_ = image.shape

  x, y, width, height = bbox
  buffered_x = x - buffer

  if buffered_x < 0:
    buffered_x = 0

  buffered_y = y - buffer

  if buffered_y < 0:
    buffered_y = 0

  buffered_height = height + 2*buffer #2 buffers, one for top and one for bottom (to adjust for the x, y change)

  if buffered_height + buffered_y > img_height:
    buffered_height = img_height - buffered_y #maximum it can be without going off image

  buffered_width = width + 2*buffer

  if buffered_width + buffered_x > img_width:
    buffered_width = img_width - buffered_x

  return (buffered_x,buffered_y,buffered_width,buffered_height)

In [ ]:
def add_regularisation_border(image,desired_width,desired_height,landmarks,image_path):
  #add a border to an image such that it becomes the same size as is desired

  """
  Assistance: https://stackoverflow.com/questions/43391205/add-padding-to-images-to-get-them-into-the-same-shape
  """

  img_height, img_width, img_channels = image.shape #channels should be constant
  color = (0,0,0) #black border
  new_image = np.full((desired_height,desired_width,img_channels), color, dtype=np.int32) #just creates a block coloured image of desired size.
  
  #centre offset
  xx = (desired_width - img_width) // 2
  yy = (desired_height - img_height) // 2

  # copy original image into centre of block-coloured image
  new_image[yy:yy+img_height, xx:xx+img_width] = image

  #As an extra, we need to adjust the landmarks such that they are not thrown off by the additional image size
  image_name = str.split(image_path,'/')[-1]
  image_name = str.split(image_name,'.')[0]
  try:
    current_landmarks = landmarks[image_name]
  except KeyError:
    print("THIS IMAGE HAS NO LANDMARK DATA!!!!!!!!!!!!!!!",image_name)

    return None, landmarks, 1 #final value is the error decider
 
  #now adjust the landmarks
  for i, point in enumerate(current_landmarks):

    landmarks[image_name][i][0] += xx
    landmarks[image_name][i][1] += yy

  return new_image, landmarks, 0

In [ ]:
def Count_Dictionary_Values(dictionary):
  #check how many keys in a dict have the same value
  new_dict = {}
  for key,value in dictionary.items():
    if value not in new_dict:
      new_dict[value] = 1
    else:
      new_dict[value] += 1
  return new_dict

In [ ]:
#We have multiple jsons and hence w emust go through them iteratively.

list_of_jsons = glob.glob('/content/New_Databases_August/Raw_Data/*.json')
%cd '/content/New_Databases_August/Raw_Data/images'

!mkdir 'individual_teeth'
!mkdir 'individual_teeth/single_roots'
!mkdir 'individual_teeth/double_roots'
!mkdir 'individual_teeth/triple_roots'

error_x_rays = {}
bboxes = {}
landmarks = {}
insufficient_landmarks = {}
for json_path in list_of_jsons:
  with open(json_path) as json_file:
    data = json.load(json_file)

  try:
    image_list = data['_via_img_metadata']
  except KeyError: #the jsons are not in the same format
    image_list = data

  teeth_in_image = 0
  amount_of_unassigned_points = 0
  non_tooth_ids = 0
  buffer = 25 #buffer for the bounding boxes

  #examples of errors below:

  # example_image = image_list['002 copy.jpg111200'] #IMPORTANT: This shows the error oin the bounding box in the dataset
  # example_image = image_list['051 copy 8.jpg106811'] #IMPORTANT: Single_2 on this image only has one landmark - confirmed in data too

  for _,example_image in image_list.items():
    image_name = example_image['filename']
    full_img = img = cv2.imread(image_name)

    example_image_regions = example_image['regions']
    example_image_attributes = example_image['file_attributes']

    #Find the bone level to make a proper mask!
    for datapoint in example_image_regions:
      shape_attr = datapoint['shape_attributes'] #contains bounding box and point locations
      region_attr = datapoint['region_attributes']

      if not region_attr:
        continue
      elif 'Toothtype' not in region_attr:
        continue #unassigned points ?
      tooth_type = region_attr['Toothtype']
      # print("tooth_type",tooth_type)
      if tooth_type == 'Bone_region':
        try:
          x_points = shape_attr['all_points_x']
        except KeyError:
          error_x_rays[image_name] = "Labelled bone level but isn't" #wrong tooth type - labelled bone level but it's not
          continue
        y_points = shape_attr['all_points_y']

        for i in range(len(x_points)):
          current_x_y = [[x_points[i],y_points[i]]] #the points / contours must be in this form
          if i == 0:
            bone_level_points = current_x_y
          else:
            bone_level_points = np.vstack((bone_level_points,current_x_y))

    for datapoint in example_image_regions:
      shape_attr = datapoint['shape_attributes'] #contains bounding box and point locations
      region_attr = datapoint['region_attributes'] #contains tooth type and tooth id

      if not region_attr: #if dict is empty
        amount_of_unassigned_points += 1
        #Some of the datapoints don't have any "single" or "double" root etc,
        #there are just arbitrary points with no assignment
        continue

      elif 'Tooth_ID' not in list(region_attr.keys()):
        #this is common and expected when the "bone region" is labelled
        continue

      tooth_ID = region_attr['Tooth_ID'] #assigns this particular datapoint to a certain tooth
      tooth_type = str.split(tooth_ID,'_')[0]#single, double or triple - for saving into the right folder!

      """
      So what we would like to do is crop the images so that each one only
      contains one tooth firstly. After this, creating a simple dictionary
      (to preface a dataframe) which has each image and tooth as a key and their
      data laid out well would be good.

      Help with bounding box cropping: 
      https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python
      """
      datapoint_name = shape_attr['name']

      """
      Cropping and masking the images
      """
      
      if datapoint_name == 'polygon' or datapoint_name == 'polyline': #begin the cropping process

        teeth_in_image += 1 #this should be printed after each image analysis, for manual checks

        #firstly turn the "all points x" and "all points y" into real point array
        all_x_points = shape_attr['all_points_x']
        all_y_points = shape_attr['all_points_y']
        amount_of_bbox_points = len(all_x_points) #same size as y points

        for i in range(amount_of_bbox_points):
          current_x_y = [[all_x_points[i],all_y_points[i]]] #the points / contours must be in this form
          if i == 0:
            regularised_bbox_points = current_x_y
          else:
            regularised_bbox_points = np.vstack((regularised_bbox_points,current_x_y))  

        #Next: Establish the rectangular bbox as images must be rectangular (img dimensions)
        rect_bbox = cv2.boundingRect(regularised_bbox_points)

        new_rect_bbox = add_smart_buffer(full_img,rect_bbox,buffer)
        x,y,width,height = new_rect_bbox #REASSIGN!
        #x,y is "bottom left" corner and width and height can find all other corners

        """
        It may be more useful to "black out" any of the teeth which are not
        the tooth in question - whilt retaining the bone level !
        mask both tooth and bone level before cropping!
        """

        #two separate masks below
        tooth_mask = np.zeros(full_img.shape[:2], np.uint8)
        cv2.drawContours(tooth_mask, [regularised_bbox_points], -1, (255, 255, 255), -1, cv2.LINE_AA)

        bone_mask = np.zeros(full_img.shape[:2], np.uint8)
        cv2.drawContours(tooth_mask, [bone_level_points], -1, (255, 255, 255), -1, cv2.LINE_AA)


        tooth_and_bone_mask = tooth_mask + bone_mask #overall mask

        # masked_tooth = full_img
        masked_tooth = cv2.bitwise_and(full_img, full_img, mask=tooth_and_bone_mask)

        cropped_single_tooth = masked_tooth[y:y+height, x:x+width].copy() #crop the masked version. not full_img
        #now we have a copy of a single cropped image of one tooth

        #Now we have succesffuly cropped the tooth!

        #If the bounding box only outlines the tooth, then adding a mask would
        #get rid of the bone segment - I should consult Sophia & Anastasiya about
        #whether or not this could hiner the network (perhaps show both options?)
        

        #For saving, firstly let's create a new name for the singular tooth image
        #Just the old name with the tooth_ID appended
        image_name = str.split(image_name,'.')[0] # get rid of the .jpg
        new_name = image_name + '_' + tooth_ID + '.jpg'

        bboxes[str.split(new_name,'.')[0]] = new_rect_bbox #need to record bboxes for the point translation
        if tooth_type == 'Single':
          cv2.imwrite('individual_teeth/single_roots/' + new_name, cropped_single_tooth)
        elif tooth_type == 'Double':
          cv2.imwrite('individual_teeth/double_roots/' + new_name, cropped_single_tooth)
        elif tooth_type == 'Triple':
          cv2.imwrite('individual_teeth/triple_roots/' + new_name, cropped_single_tooth)
        else:
          raise Exception("Unidentified Tooth Type")
      teeth_in_image -= 1 #one of the polygons is bone

  #if datapoint isn't polygon - it should be a point - loop through again since we need all bboxes first!!
    for datapoint in example_image_regions:
      shape_attr = datapoint['shape_attributes'] #contains bounding box and point locations
      region_attr = datapoint['region_attributes'] #contains tooth type and tooth id

      if not region_attr:
        amount_of_unassigned_points += 1
        continue
      elif 'Tooth_ID' not in list(region_attr.keys()):
        non_tooth_ids += 1
        continue

      tooth_ID = region_attr['Tooth_ID'] 
      tooth_type = str.split(tooth_ID,'_')[0]#single, double or triple
      datapoint_name = shape_attr['name']

      if datapoint_name != 'polygon' and datapoint_name != 'polyline': #points, not polygons

        """
        Creating the CSV to show the points on the cropped images

        I have spent a little bit of time investigating in what form to create my dataset,
        however either way, when it comes to training the set would need to be flattened
        out and regressed upon, therefore we may as well just flatten it out and put it into
        a CSV. Usually distributable datasets are in .pts format bt this is not entirely necessary here.
        """

        #start it off with a dictionary with the teeth names in.
        image_name = str.split(image_name,'.')[0] # get rid of the .jpg
        new_name = image_name + '_' + tooth_ID

        # if (new_name == '003_Single_1' or new_name == '009 copy 5_Single_2' or new_name == '009 copy_Single_1' #IMPORTANT: ERRORS IN DATASET
        # or new_name == '029 copy 6_Double_2'):
        #   print("ERROR!!!!!!!!!!!!")
        #   continue
        # print("datapoint name",datapoint_name)
        x_point = shape_attr['cx'] #x value of this paticular point
        y_point = shape_attr['cy'] #y value of this paticular point

        try:
          tooth_bbox = bboxes[new_name] #the bounding box should always be calculated before the point analysis!
        except KeyError:
          error_x_rays[new_name] = "no bbox" #no bounding box
          continue
        # print("tooth bbox",tooth_bbox)
        bbox_x,bbox_y,_,_ = tooth_bbox
        new_x_point = x_point - bbox_x #x point after accounting for the cropped image !!
        new_y_point = y_point - bbox_y #y point after accounting for the cropped image !!
        


        #find the current image to print the point on it:
        # print("new name",new_name)
        if tooth_type == 'Single':
          current_tooth = cv2.imread('individual_teeth/single_roots/' + new_name + '.jpg') #TODO: HOW DOES THIS WORK IF THE IMAGE IS A .TIF ??
        elif tooth_type == 'Double':
          current_tooth = cv2.imread('individual_teeth/double_roots/' + new_name + '.jpg')
        elif tooth_type == 'Triple':
          current_tooth = cv2.imread('individual_teeth/triple_roots/' + new_name + '.jpg')

        show_points = cv2.circle(current_tooth, (new_x_point,new_y_point), 4, color=(0, 0, 255))
        # cv2_imshow(show_points)

        """
        Determine the type of the landmark!
        """

        if tooth_type == 'Single':
          try:
            landmark_type = region_attr['Single_root']
          except KeyError:
            landmark_type = None #this means that there is an error 
        elif tooth_type == 'Double':
          try: 
            landmark_type = region_attr['Double_root']
          except KeyError:
            landmark_type = None #this means that there is an error 
        elif tooth_type == 'Triple':
          try: 
            landmark_type = region_attr['Triple_root']
          except KeyError:
            landmark_type = None #this means that there is an error 
        else:
          print("ERROR ON TOOTH TYPE")
          break

        if new_name not in list(landmarks.keys()) and new_name not in list(error_x_rays.keys()):
          """
          In order to posit a discrete order of landmarks, we should first create a
          system whereby all of the landmarks have their own order, and it can be identified
          if they're not in that order
          """
          #create the placeholder

          if tooth_type == 'Single':
            filling_matrix = np.empty((ideal_landmarks_single,2))
            filling_matrix.fill(np.nan) #creates a matrix full of NaN results
            landmarks[new_name] = filling_matrix
          elif tooth_type == 'Double':
            filling_matrix = np.empty((ideal_landmarks_double,2))
            filling_matrix.fill(np.nan) #creates a matrix full of NaN results
            landmarks[new_name] = filling_matrix
          elif tooth_type == 'Triple':
            filling_matrix = np.empty((ideal_landmarks_triple,2))
            filling_matrix.fill(np.nan) #creates a matrix full of NaN results
            landmarks[new_name] = filling_matrix
          else:
            print("ERROR ON TOOTH TYPE")
            break

        if landmark_type == None: #there is a faulty landmark so the data can't be properly analysed
          error_x_rays[new_name] = "landmark type is none"
          if new_name in list(landmarks.keys()): #the same tooth can have multiple labelling errors
            del landmarks[new_name]

        else:

          """
          Now that we have made sure that all of the teeth have landmark placeholders,
          we can fill them out.
          """
          new_point = [new_x_point]
          new_point.append(new_y_point)
          if tooth_type == 'Single':
            landmark_order = single_landmark_order[landmark_type]
          elif tooth_type == 'Double':
            try: 
              landmark_order = double_landmark_order[landmark_type]
            except KeyError:
              #IMPORTANT: there are some incorrectly labelled x-rays and hence these cannot be analysed... they are labelled as both single and double
              error_x_rays[new_name] = "error ??" #so that these can be ignored (and hopefully fixed at a later date ??)
              if new_name in list(landmarks.keys()): #the same tooth can have multiple labelling errors
                del landmarks[new_name]
              # sys.exit()
          elif tooth_type == 'Triple':
            landmark_order = triple_landmark_order[landmark_type]
          else:
            print("ERROR ON TOOTH TYPE")
            break
        
        if new_name not in list(error_x_rays.keys()):
          landmarks[new_name][landmark_order] = new_point

"""
Now we must make sure that all images are the same size - thus resave them!
we need to loop through all of the cropped images, find their size and regularise them
"""
image_list_single = glob.glob('individual_teeth/single_roots/*.jpg')
image_list_double = glob.glob('individual_teeth/double_roots/*.jpg')
image_list_triple = glob.glob('individual_teeth/triple_roots/*.jpg')

image_list = image_list_single
image_list.extend(image_list_double)
image_list.extend(image_list_triple)

for i, image_path in enumerate(image_list): #delete the teeth with labelling errors
  image_name = str.split(image_path,'/')[-1]
  image_name = str.split(image_name,'.')[0]
  if image_name in list(error_x_rays.keys()):
    del image_list[i]

max_height = 0 #instantiation
max_width = 0
for image_path in image_list:
  original_img = cv2.imread(image_path)
  img_height, img_width,_ = original_img.shape
  if img_height > max_height:
    max_height = img_height
  if img_width > max_width:
    max_width = img_width

for image_path in image_list:
  # sys.exit()
  img = cv2.imread(image_path)
  img_name = str.split(image_path,'/')[-1]
  img_name = str.split(img_name,'.')[0]
  try:
    landmark_list = landmarks[img_name] #shape: number of landmarks x 2
  except KeyError:
    error_x_rays[img_name] = "no landmarks"
    continue
  for point in landmark_list: #point is 2-length list
    if point[0] != point[0]: #great way to check for NaNs
      insufficient_landmarks[img_name] = img_name
      continue #this shows that the image does not have all the required landmarks

    current_x = int(point[0]) #todo: change landmark array form to int instead of float - would this compromise anything ?
    current_y = int(point[1])
    visualise_img = cv2.circle(img, (current_x,current_y), 4, color=(0, 0, 255))

  print(img_name,":")
  cv2_imshow(visualise_img)

amount_of_teeth = len(list(landmarks.keys()))
amount_of_errors = len(list(error_x_rays.keys()))

# print("landmarks: ",landmarks)
print("landmarks keys",list(landmarks.keys()))
print("error x rays",error_x_rays)
print("amount of error x rays",amount_of_errors)
print("amount of usable x rays",amount_of_teeth)
print("percentage of error x rays: ",(amount_of_errors*100)/(amount_of_errors+amount_of_teeth),"%")
print("types of errors",Count_Dictionary_Values(error_x_rays))
print("amount_of_unassigned_points",amount_of_unassigned_points)
print("non_tooth_ids",non_tooth_ids)

Now I use the landmarks dictionary to make CSVs

In [ ]:
#we need 3 CSVs - for single, double and triple teeth, since they have different numbers of landmarks
single_counter = 0 #instantiations
double_counter = 0
triple_counter = 0

single_correct_landmarks = 0
double_correct_landmarks = 0
triple_correct_landmarks = 0
V = ideal_landmarks_single
single_correct_length = 5*2 + 1
double_correct_length = 8*2 + 1 
triple_correct_length = 9*2 + 1 

#manually seen from a look through the above visualised radiographs
visually_incorrect_labels = ['009 copy 5_Single_1', '051 copy 8_Single_1',
                             '051 copy 7_Single_1', '051 copy 7_Single_2',
                             '015 copy 2_Single_1']

for i, (key, value) in enumerate(landmarks.items()):
  
  """
  So from visual inspection, there are more labelling database errors which involve
  some teeth being labelled by their boundign box as one tooth, but the landmarks are assigned
  to another - these should be excluded from the training set as they could heavily jeopardise
  the training.
  """
  if key in visually_incorrect_labels:
    print(key,"has errors in its labelling and hence is being disregarded")
    error_x_rays[key] = "visually incorrect"
    continue #just skip this one

  #currently the goal is to disregard all images which do not have the desired
  #number of landmarks annotated - therefore, they will simply not be included in the csv
  all_landmarks_present = 1
  for point in value:
    if point[0] != point[0] or point[1] != point[1]: #checking for NaN
      all_landmarks_present = 0
  
  if all_landmarks_present == 0:
    error_x_rays[key] = "not all landmarks labelled"
    continue #skip the rest of this loop - do not record the tooth image data

  root_type = str.split(key,'_')[-2]

  key_list = [key]
  value = np.concatenate(value).ravel().tolist() #full of numpy arrays - unravelling them.
  key_list.extend(value)
  csv_row = np.array(key_list)

  if root_type == 'Single':
    single_counter += 1
    if single_counter == 1:
      csv_single_matrix = csv_row
    else:
      csv_single_matrix = np.vstack((csv_single_matrix,csv_row))


    if len(csv_row) == single_correct_length:
      single_correct_landmarks += 1
  if root_type == 'Double':
    double_counter += 1
    if double_counter == 1:
      csv_double_matrix = csv_row
    else:
      csv_double_matrix = np.vstack((csv_double_matrix,csv_row))

    
    if len(csv_row) == double_correct_length:
      double_correct_landmarks += 1


  if root_type == 'Triple':
    triple_counter += 1
    if triple_counter == 1:
      csv_triple_matrix = csv_row
    else:
      csv_triple_matrix = np.vstack((csv_triple_matrix,csv_row))

    if len(csv_row) == triple_correct_length:
      triple_correct_landmarks += 1

print("csv single matrix",csv_single_matrix)
print("csv double matrix",csv_double_matrix)
print("csv triple matrix",csv_triple_matrix)

print("csv single matrix shape",csv_single_matrix.shape)
print("csv double matrix shape",csv_double_matrix.shape)
print("csv triple matrix shape",csv_triple_matrix.shape)

print("landmarks[051 copy 8_Single_2]",landmarks['051 copy 8_Single_2'])

print("single correct landmarks: ",single_correct_landmarks ," / ",single_counter)
print("double correct landmarks: ",double_correct_landmarks ," / ",double_counter)
print("triple correct landmarks: ",triple_correct_landmarks ," / ",triple_counter)


# amount_of_teeth = len(list(landmarks.keys()))
amount_of_teeth = single_correct_landmarks + double_correct_landmarks + triple_correct_landmarks
amount_of_errors = len(list(error_x_rays.keys()))
print("THESE ARE THE ERRORS LISTED!!!!!!!!!!!!!!!!!!!",list(error_x_rays.keys()))
print("amount of error x rays",amount_of_errors)
print("amount of usable x rays",amount_of_teeth)
print("percentage of error x rays: ",(amount_of_errors*100)/(amount_of_errors+amount_of_teeth),"%")
print("types of errors",Count_Dictionary_Values(error_x_rays))



#save the csv files
pd.DataFrame(csv_single_matrix).to_csv("individual_teeth/single_roots.csv")
pd.DataFrame(csv_double_matrix).to_csv("individual_teeth/double_roots.csv")
pd.DataFrame(csv_triple_matrix).to_csv("individual_teeth/triple_roots.csv")

#note that the printed reasoning may not be the only error that the image has it could have numeroud,
#but only have one written (to avoid repetition)

    

In [ ]:
print("error x rays",error_x_rays)
#1079_Single_1 is example

In [ ]:
!zip -r individual_teeth.zip individual_teeth